# Monte Carlo Experiment: PSR test vs Non-Central Student’s t-distribution test

**Goal:** For each target Sharpe SR₀, generate a returns-realistic (neg-skew, leptokurtic) mixture of Gaussians whose population Sharpe equals SR₀, and then evaluate PSR vs t-test against that SR₀.

**Data-Generating Process:**
- Mixture body: Normal(μ_core, σ_core); tail: Normal(μ_tail, σ_tail) with prob p_tail (negative shocks).
- To set SR₀: compute the mixture population σ at zero mean, then add a constant μ_shift = SR₀·σ to all observations. (Adding a constant preserves skewness/kurtosis, and sets E[r]/σ = SR₀.)

In [ ]:
import numpy as np, pandas as pd
from scipy import stats

# --- settings ---
REPS = 10_000
T = 252*5                       # 5y daily
SR0_annual_list = [0.0, 0.5, 1., 1.5, 2.]
SR0_list = [s/np.sqrt(252) for s in SR0_annual_list]
RSEED = 2025
# Mixture configs: (name, p_tail, mu_tail, sigma_tail, sigma_core)
configs = [
    ("gaussian", 0.00, 0.00, 0.010, 0.010),
    ("mild",     0.04, -0.03, 0.015, 0.010),
    ("moderate", 0.03, -0.045, 0.020, 0.010),
    ("severe",   0.02, -0.060, 0.025, 0.010),
]

def mixture_variance(p_tail, mu_tail, sigma_tail, mu_core, sigma_core):
    w = 1.0 - p_tail
    mu = w*mu_core + p_tail*mu_tail
    m2 = w*(sigma_core**2 + mu_core**2) + p_tail*(sigma_tail**2 + mu_tail**2)
    return m2 - mu**2

def gen_with_true_SR0(reps, T, cfg, SR0, seed):
    name, p, mu_tail, sig_tail, sig_core = cfg
    # Zero-mean baseline mixture (choose mu_core so mean=0)
    mu_core0 = - p*mu_tail/(1.0 - p)
    std0 = np.sqrt(mixture_variance(p, mu_tail, sig_tail, mu_core0, sig_core))
    mu_shift = SR0 * std0  # sets population Sharpe to SR0, preserves skew/kurt
    rng = np.random.default_rng(seed)
    mask = rng.random((reps, T)) < p
    X = rng.normal(mu_core0 + mu_shift, sig_core, size=(reps, T))
    X[mask] = rng.normal(mu_tail + mu_shift, sig_tail, size=mask.sum())
    return X

def psr_z_T(X, SR0):
    Tn = X.shape[1]
    s = X.std(axis=1, ddof=1)
    sr_hat = X.mean(axis=1)/s
    skew = stats.skew(X, axis=1, bias=False)
    kappa = stats.kurtosis(X, axis=1, fisher=True, bias=False) + 3.0
    den = np.sqrt((1.0/Tn) * (1.0 - skew*SR0 + ((kappa-1.0)/4.0)*(SR0**2)))
    return (sr_hat - SR0)/den

def t_stat(X, SR0):
    Tn = X.shape[1]
    s = X.std(axis=1, ddof=1)
    sr_hat = X.mean(axis=1)/s
    # return np.sqrt(Tn)*(sr_hat - SR0)   ################### BUG: MISSING VARIANCE (it was added below)
    skew = 0
    kappa = 3.0
    den = np.sqrt((1.0/Tn) * (1.0 - skew*SR0 + ((kappa-1.0)/4.0)*(SR0**2)))
    return (sr_hat - SR0)/den    

rows = []
for cfg in configs:
    for SR0 in SR0_list:
        X = gen_with_true_SR0(REPS, T, cfg, SR0, seed=RSEED + int(1e6*SR0) + hash(cfg[0])%10000)
        # realized moments
        avg_skew = float(np.mean(stats.skew(X, axis=1, bias=False)))
        avg_exk  = float(np.mean(stats.kurtosis(X, axis=1, fisher=True, bias=False)))
        # stats and KS
        z = psr_z_T(X, SR0)
        t = t_stat(X, SR0)
        # stats and KS (probability space)
        psr = stats.norm.cdf(z)
        ks_psr = stats.kstest(psr, 'uniform')
        p_t = stats.t.sf(t, df=T-1) # if SR0 == 0 else stats.nct.sf(t, df=T-1, nc=np.sqrt(T)*SR0)
        ks_t   = stats.kstest(p_t, 'uniform')
        rows.append({
            'config': cfg[0], 'T': T,
            'SR0_annual': SR0*np.sqrt(252),
            'avg_skew': avg_skew, 'avg_excess_kurtosis': avg_exk,
            'KS_PSR_D': float(ks_psr.statistic), 'KS_t_D': float(ks_t.statistic),
            'KS_PSR_p': float(ks_psr.pvalue), 'KS_t_p': float(ks_t.pvalue),
            'PSR_better?': float(ks_psr.statistic) < float(ks_t.statistic),
        })
df = pd.DataFrame(rows).round(6)
df.to_csv('appendix_1.csv')
df

In [ ]:
from functions import sharpe_ratio_variance
def my_psr_z_T(X, SR0, rho):
    Tn = X.shape[1]
    s = X.std(axis=1, ddof=1)
    sr_hat = X.mean(axis=1)/s
    skew = stats.skew(X, axis=1, bias=False)
    kappa = stats.kurtosis(X, axis=1, fisher=True, bias=False) + 3.0
    v = sharpe_ratio_variance( SR0, Tn, gamma3=skew, gamma4=kappa, rho=rho, K=1 )
    den = np.sqrt(v)
    return (sr_hat - SR0)/den

if False: 
    print( psr_z_T(X, SR0) )
    print( my_psr_z_T(X, SR0, 0) )  # Same values


In [ ]:
from tqdm.auto import tqdm
from itertools import product
from functions import generate_non_gaussian_data, generate_autocorrelated_non_gaussian_data
import ray

ray.init()

In [ ]:
@ray.remote
def f1(name, rho, SR0):
    if rho == 0: 
        X = generate_non_gaussian_data( T, REPS, SR0 = SR0, name = name )
    else: 
        X = generate_autocorrelated_non_gaussian_data( T, REPS, SR0 = SR0, name = name, rho = rho )

    X = X.T
    avg_skew = float(np.mean(stats.skew(X, axis=1, bias=False)))
    avg_exk  = float(np.mean(stats.kurtosis(X, axis=1, fisher=True, bias=False)))
    # stats and KS
    z = my_psr_z_T(X, SR0, rho)
    t = t_stat(X, SR0)
    # stats and KS (probability space)
    psr = stats.norm.cdf(z)
    ks_psr = stats.kstest(psr, 'uniform')
    p_t = stats.t.sf(t, df=T-1) # if SR0 == 0 else stats.nct.sf(t, df=T-1, nc=np.sqrt(T)*SR0)
    assert len(p_t) == REPS
    ks_t   = stats.kstest(p_t, 'uniform')
    return {
        'name': name,
        'rho': rho,
        'SR0_annual': SR0*np.sqrt(252),
        'avg_skew': avg_skew, 'avg_excess_kurtosis': avg_exk,
        'KS_PSR_D': float(ks_psr.statistic), 'KS_t_D': float(ks_t.statistic),
        'KS_PSR_p': float(ks_psr.pvalue), 'KS_t_p': float(ks_t.pvalue),
        'PSR_better?': float(ks_psr.statistic) < float(ks_t.statistic),
    }

rows = []
RHOs = [0, .2]
rows = [ 
    f1.remote(name, rho, SR0) 
    for name, rho, SR0 in product( ['gaussian', 'mild', 'moderate', 'severe'], RHOs, SR0_list )
]
rows = [ ray.get(r) for r in tqdm(rows) ]
rows = pd.DataFrame( rows )

In [ ]:
d = rows.copy()
d['avg_skew'] = d['avg_skew'].round(1)
d['avg_excess_kurtosis'] = d['avg_excess_kurtosis'].round(1)
d['Diff'] = d['KS_t_D'] - d['KS_PSR_D']
d['KS_PSR_D'] = d['KS_PSR_D'].round(3)
d['KS_t_D'] = d['KS_t_D'].round(3)
d['KS_PSR_p'] = d['KS_PSR_p'].round(3)
d['KS_t_p'] = d['KS_t_p'].round(3)
d['Diff'] = d['Diff'].round(3)
d.columns = ['Distribution', 'ρ', 'Annual SR0', 'Avg Skew', 'Avg Ex. Kurt', 'KS_PSR', 'KS_t', 'p(KS_PSR)', 'p(KS_t)', 'PSR better?', 'Diff']
d.drop(columns=['PSR better?'], inplace=True)
d.to_csv('exhibit_1bis.csv', index = False)

In [ ]:
from IPython.display import display

def highlight_pos_neg(val):
    """
    Highlight negative values in light red, positive in light green.
    """
    color = ''
    try:
        v = float(val)
        if v < 0:
            color = 'background-color: #ffd6d6'   # light red
        elif v > 0:
            color = 'background-color: #d6ffd6'   # light green
    except:
        color = ''
    return color

d_fmt = d.copy()
for col in d_fmt.columns[1:5]:
    d_fmt[col] = d_fmt[col].astype(float).map(lambda x: f'{x:.1f}')
for col in d_fmt.columns[5:]:
    d_fmt[col] = d_fmt[col].astype(float).map(lambda x: f'{x:.3f}')
styled = d_fmt.style.map(highlight_pos_neg, subset=[d.columns[-1]])

def add_hr(styler, rows=[4,9,14,19,24,29,34]):
    # Create empty DataFrame of same shape to hold style strings
    styles = pd.DataFrame("", index=styler.index, columns=styler.columns)
    # Apply a thick border to all columns in the specified row
    for row in rows:
        if row in styles.index:
            styles.loc[row, :] = "border-bottom: 2px solid black;"
    return styles

styled = styled.apply(add_hr, axis=None)

display(styled)

# Precision and Recall of PSR

In [ ]:
# SR0 = 0, SR1 ∈ {0.5, 1.0, 1.5, 2.0}

SR0_annual = 0.0
SR0_daily  = 0.0
SR1_annual_list = [0.5, 1.0, 1.5, 2.0]
SR1_daily_list  = [s/np.sqrt(252) for s in SR1_annual_list]

def _confusion_metrics(y_true, pvals, alpha=0.05):
    yhat = (pvals < alpha)
    TP = int(((y_true==1)&(yhat)).sum())
    FP = int(((y_true==0)&(yhat)).sum())
    TN = int(((y_true==0)&(~yhat)).sum())
    FN = int(((y_true==1)&(~yhat)).sum())
    prec = TP/(TP+FP) if (TP+FP)>0 else np.nan
    rec  = TP/(TP+FN) if (TP+FN)>0 else np.nan
    f1   = (2*prec*rec)/(prec+rec) if (prec>0 and rec>0) else (0.0 if (prec==0 or rec==0) else np.nan)
    return prec, rec, f1

rows = []
for cfg in configs:
    for SR1_daily, SR1_annual in zip(SR1_daily_list, SR1_annual_list):
        # Null: SR = 0 ; Alternative: SR = SR1 (annual)
        X0 = gen_with_true_SR0(REPS, T, cfg, SR0=SR0_daily, seed=RSEED)
        X1 = gen_with_true_SR0(REPS, T, cfg, SR0=SR1_daily, seed=RSEED+1)
        y_true = np.r_[np.zeros(len(X0), dtype=int), np.ones(len(X1), dtype=int)]

        # PSR one-sided test (H1: SR > 0)
        p_psr = np.r_[stats.norm.sf(psr_z_T(X0, SR0_daily)),
                      stats.norm.sf(psr_z_T(X1, SR0_daily))]
        prec, rec, f1 = _confusion_metrics(y_true, p_psr, alpha=0.05)

        rows.append({
            "config": cfg[0],
            "SR1_annual": SR1_annual,
            "PSR_precision": prec,
            "PSR_recall": rec,
            "PSR_F1": f1
        })

psr_table = pd.DataFrame(rows).sort_values(
    ["config","SR1_annual"]
).set_index(["config","SR1_annual"]).round(4)
psr_table.to_csv('appendix_2.csv')
psr_table

In [ ]:
@ray.remote
def f2(rho, name, SR0_daily, SR1_daily, SR1_annual):
    # Null: SR = 0 ; Alternative: SR = SR1 (annual)
    if rho == 0:
        X0 = generate_non_gaussian_data( T, REPS, SR0 = SR0_daily, name = name )
        X1 = generate_non_gaussian_data( T, REPS, SR0 = SR1_daily, name = name )
    else:
        X0 = generate_autocorrelated_non_gaussian_data( T, REPS, SR0 = SR0_daily, name = name, rho = rho )
        X1 = generate_autocorrelated_non_gaussian_data( T, REPS, SR0 = SR1_daily, name = name, rho = rho )
            
    y_true = np.r_[np.zeros(REPS, dtype=int), np.ones(REPS, dtype=int)]

    # PSR one-sided test (H1: SR > 0)
    p_psr = np.r_[
        stats.norm.sf(my_psr_z_T(X0.T, SR0_daily, rho)),
        stats.norm.sf(my_psr_z_T(X1.T, SR0_daily, rho)),
    ]
    prec, rec, f1 = _confusion_metrics(y_true, p_psr, alpha=0.05)

    return {
        "name": name,
        "rho": rho, 
        "SR1_annual": SR1_annual,
        "PSR_precision": prec,
        "PSR_recall": rec,
        "PSR_F1": f1
    }

rows = [
    f2.remote(rho, name, SR0_daily, SR1_daily, SR1_annual)
    for rho in RHOs
    for name in ['gaussian', 'mild', 'moderate', 'severe']
    for SR1_daily, SR1_annual in zip(SR1_daily_list, SR1_annual_list)
]
rows = [ ray.get(r) for r in tqdm(rows) ]
rows = pd.DataFrame(rows)

In [ ]:
rho = 0
name = "gaussian"
SR0_daily = 0
SR1_daily = SR1_daily_list[1]
SR1_annual = SR1_annual_list[1]

X0 = generate_non_gaussian_data( T, REPS, SR0 = SR0_daily, name = name )
X1 = generate_non_gaussian_data( T, REPS, SR0 = SR1_daily, name = name )
    
y_true = np.r_[np.zeros(REPS, dtype=int), np.ones(REPS, dtype=int)]

p_psr = np.r_[
    stats.norm.sf(my_psr_z_T(X0.T, SR0_daily, rho)),
    stats.norm.sf(my_psr_z_T(X1.T, SR0_daily, rho)),
]
prec, rec, f1 = _confusion_metrics(y_true, p_psr, alpha=0.05)
rec

In [ ]:
psr_table = pd.DataFrame(rows).sort_values(
    ["name","rho","SR1_annual"]
).set_index(["name","rho","SR1_annual"]).round(3)
psr_table.to_csv('exhibit_2bis.csv')
psr_table